# C) Privacy DSAR
Process DSAR intake, verify identity, export access packages or prepare delete previews, and log everything.

- Load CSV datasets (customers, intake)
- Filter data by requester email
- Package results (ZIP) for access
- Record a processing log with timestamps


In [2]:
from pathlib import Path
import pandas as pd, zipfile, datetime
BASE = Path('..').resolve()
print('Base path:', BASE)

Base path: C:\Project\Miss Shoes\01_Case\C_Privacy_DSAR


In [3]:
# Load datasets
customers = pd.read_csv(BASE/'data'/'customers.csv')
intake = pd.read_csv(BASE/'data'/'dsar_intake.csv')
customers.head(), intake.head()

(   customer_id        full_name                         email  \
 0            1    Taylor Garcia    taylor.garcia1@example.com   
 1            2    Jordan Garcia       jordan.garcia2@mail.com   
 2            3      Casey Lopez      casey.lopez3@shoebox.com   
 3            4  Taylor Martinez  taylor.martinez4@example.com   
 4            5    Dana Anderson       dana.anderson5@mail.com   
 
              phone               address last_order_date  loyalty_points  \
 0  +1-555-926-5984  856 Main St, Apt 722      2025-05-20              41   
 1  +1-555-354-6828  415 Main St, Apt 526      2022-04-11             693   
 2  +1-555-975-8408  579 Main St, Apt 767      2025-01-31             255   
 3  +1-555-112-2010  860 Main St, Apt 800      2023-04-01             133   
 4  +1-555-745-2376  606 Main St, Apt 601      2023-06-15             939   
 
    marketing_opt_in  created_at  
 0              True  2022-11-03  
 1             False  2022-06-17  
 2              True  2023-08-05 

In [4]:
# Process requests
from pathlib import Path
out = BASE/'outputs'; out.mkdir(exist_ok=True, parents=True)
log = []
for _,c in intake.iterrows():
    row = {'case_id':c['case_id'],'type':c['request_type'],'email':c['requester_email'],'verified':c['identity_verified']}
    if str(c['identity_verified']).lower()!='yes':
        row['status']='rejected_identity_not_verified'; log.append(row); continue
    subset = customers[customers.email==c['requester_email']]
    ts = datetime.datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
    if c['request_type']=='access':
        if subset.empty:
            row['status']='no_records_found'
        else:
            csv_name = f"customers_{c['case_id']}.csv"
            subset.to_csv(out/csv_name, index=False)
            zname = out/f"dsar_{c['case_id']}.zip"
            with zipfile.ZipFile(zname,'w',zipfile.ZIP_DEFLATED) as zf:
                zf.write(out/csv_name, arcname=csv_name)
            row['status']='fulfilled_access'; row['artifact']=zname.name
    else:  # delete
        if subset.empty:
            row['status']='no_records_found'
        else:
            prev = out/f"delete_preview_{c['case_id']}.csv"
            customers[customers.email!=c['requester_email']].head(3).to_csv(prev, index=False)
            row['status']='prepared_delete_preview'; row['artifact']=prev.name
    row['processed_at']=ts
    log.append(row)
import pandas as pd
log_df = pd.DataFrame(log)
log_df.head(10)

C:\Users\B_user\AppData\Local\Temp\ipykernel_4040\3118066400.py:10: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')


,case_id,type,email,verified,status,artifact,processed_at
0,DSAR-20250911-01,delete,casey.lopez3@shoebox.com,no,rejected_identity_not_verified,NaN,NaN
1,DSAR-20250911-02,access,shawn.walker153@example.com,yes,fulfilled_access,dsar_DSAR-20250911-02.zip,2025-09-20T04:50:59Z
2,DSAR-20250911-03,delete,drew.clark141@shoebox.com,yes,prepared_delete_preview,delete_preview_DSAR-20250911-03.csv,2025-09-20T04:51:00Z
3,DSAR-20250911-04,delete,robin.brown106@mail.com,no,rejected_identity_not_verified,NaN,NaN
4,DSAR-20250911-05,access,drew.wilson64@shoebox.com,yes,fulfilled_access,dsar_DSAR-20250911-05.zip,2025-09-20T04:51:00Z
5,DSAR-20250911-06,delete,kai.harris189@mail.com,no,rejected_identity_not_verified,NaN,NaN
6,DSAR-20250911-07,delete,shawn.thomas57@mail.com,yes,prepared_delete_preview,delete_preview_DSAR-20250911-07.csv,2025-09-20T04:51:00Z
7,DSAR-20250911-08,access,sasha.lewis109@mail.com,yes,fulfilled_access,dsar_DSAR-20250911-08.zip,2025-09-20T04:51:00Z
8,DSAR-20250911-09,delete,kai.davis196@example.com,no,rejected_identity_not_verified,NaN,NaN
9,DSAR-20250911-10,delete,dana.thomas167@mail.com,no,rejected_identity_not_verified,NaN,NaN


In [6]:
# Save processing log
out_log = out/'dsar_log_evidence.csv'
log_df.to_csv(out_log, index=False)
out_log

WindowsPath('C:/Project/Miss Shoes/01_Case/C_Privacy_DSAR/outputs/dsar_log_evidence.csv')